## Importação das Bibiliotecas

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

## Criando uma SparkSession

In [2]:
spark = SparkSession.builder.appName("ReadExcel").getOrCreate()


## Processo de Leitura do Excel com Dime referente ao Dados da Escola



In [5]:
#Ler o arquivo em Excel em Pandas, devido o spark não ter uma função nativda
arqEnem = 'E:\Estudos\SQL\Dime\DIME_ENEM_PARTICIPANTES.xlsx'

# Ler a aba DIME_SEXO no arquivo Excel
dfDadosGeraisDependenciaAdministrativaPD = pd.read_excel(arqEnem, sheet_name='DIME_DEPENDENCIA_ADM_ESC')

# Ler a aba DIME_FAIXA_ETARIA no arquivo Excel
dfDadosGeraisLocalizacaoPD = pd.read_excel(arqEnem, sheet_name='DIME_LOCALIZACAO_ESC')

# Ler a aba DIME_RACA no arquivo Excel
dfDadosGeraisSituacaoFuncionamentoPD = pd.read_excel(arqEnem, sheet_name='DIME_SIT_FUNC_ESC')

In [4]:
print(dfDadosGeraisPD)

  TP_SEXO_ID TP_SEXO_DESCRICAO
0          M         Masculino
1          F         Feminino 


In [10]:
#Foi necessário configurar este Path para realizar a conversão
import os

print(os.environ.get('PYSPARK_PYTHON'))


E:\anaconda3\python.exe


In [6]:
# Se você já tem um DataFrame do Pandas, pode converter diretamente para um DataFrame do Spark usando o método createDataFrame do SparkSession
dfDadosGeraisDependenciaAdministrativaSP = spark.createDataFrame(dfDadosGeraisDependenciaAdministrativaPD)
dfDadosGeraisLocalizacaoSP               = spark.createDataFrame(dfDadosGeraisLocalizacaoPD)
dfDadosGeraisSituacaoFuncionamentoSP     = spark.createDataFrame(dfDadosGeraisSituacaoFuncionamentoPD)

In [7]:
dfDadosGeraisDependenciaAdministrativaSP = dfDadosGeraisDependenciaAdministrativaSP.fillna(-2) # Preenche com -2(sem ref), por exemplo
dfDadosGeraisLocalizacaoSP               = dfDadosGeraisLocalizacaoSP.fillna(-2)               # Preenche com -2(sem ref), por exemplo
dfDadosGeraisSituacaoFuncionamentoSP     = dfDadosGeraisSituacaoFuncionamentoSP.fillna(-2)     # Preenche com -2(sem ref), por exemplo

In [63]:
df = [
dfDadosGeraisDependenciaAdministrativaSP,
dfDadosGeraisLocalizacaoSP,
dfDadosGeraisSituacaoFuncionamentoSP]

# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [12]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [64]:
# Nome das tabelas que serão criadas

Tabelas = [
    "DimeDependenciaAdministrativa",
    "DimeLocalizacao",
    "DimeSituacaoFuncionamento"
]


In [84]:
# Configurações do banco de dados em ODBC para dropar
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")
i = 0
while i < len(Tabelas):
# Crie um cursor
    cursor = conn.cursor()
    Tabela_atual = Tabelas[i]
# Execute a operação DROP TABLE na tabela 'Participantes'
    truncate_query = f"IF OBJECT_ID('{Tabela_atual}', 'U') IS NOT NULL DROP TABLE {Tabela_atual}"
    cursor.execute(truncate_query)
    #print(truncate_query)
# Commit as alterações no banco de dados
    conn.commit()
    i += 1
          

IF OBJECT_ID('DimeSexo', 'U') IS NOT NULL DROP TABLE DimeSexo
IF OBJECT_ID('DimeFaixaEtaria', 'U') IS NOT NULL DROP TABLE DimeFaixaEtaria
IF OBJECT_ID('DimeRaca', 'U') IS NOT NULL DROP TABLE DimeRaca
IF OBJECT_ID('DimeNacionalidade', 'U') IS NOT NULL DROP TABLE DimeNacionalidade
IF OBJECT_ID('DimeSituacaoConclusao', 'U') IS NOT NULL DROP TABLE DimeSituacaoConclusao
IF OBJECT_ID('DimeAnoConclusao', 'U') IS NOT NULL DROP TABLE DimeAnoConclusao
IF OBJECT_ID('DimeEscola', 'U') IS NOT NULL DROP TABLE DimeEscola
IF OBJECT_ID('DimeEnsino', 'U') IS NOT NULL DROP TABLE DimeEnsino
IF OBJECT_ID('DimeTreineiro', 'U') IS NOT NULL DROP TABLE DimeTreineiro


In [98]:
# Escreva os novos dados nas Dimes usando a conexão JDBC

i = 0
while i < len(Tabelas):
    Tabela_Atual = Tabelas[i]
    df_atual     = df[i]    
    df_atual.write \
                .format("jdbc") \
                .option("url", url) \
                .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
                .option("ssl", "true")\
                .option("trustServerCertificate", "true")\
                .option("dbtable", Tabela_Atual) \
                .option("user", username) \
                .option("password", password)\
                .option("SaveMode", "overWrite").save()
    i += 1